# 1.2 信息比率 (Information Ratio)

## 概念介绍

信息比率是一种风险调整收益指标，用于衡量投资组合相对于基准指数的超额收益与超额风险之比。信息比率衡量了投资经理在承担主动风险的情况下产生的超额收益能力。

### 信息比率公式

$Information\ Ratio = \frac{R_p - R_b}{\sigma(R_p - R_b)}$

其中：
- $R_p$ 是投资组合的收益率
- $R_b$ 是基准指数的收益率
- $\sigma(R_p - R_b)$ 是投资组合相对于基准的超额收益率的标准差（跟踪误差）

信息比率越高，表示投资经理在单位主动风险下产生的超额收益越高，意味着投资策略相对于基准指数的表现越好。通常，信息比率大于0.4被认为表现良好，大于0.6被认为表现非常好。

## 1.1 安装必要的库

首先安装需要用到的Python库

In [ ]:
# 安装必要的库
!pip install pandas numpy matplotlib

## 1.2 从Github加载数据

In [ ]:
!wget -O AAPL_data.csv "https://raw.githubusercontent.com/Newzil-git/Financial-Risk-Management/main/FR%20Code/Part%201/AAPL_data.csv"
!wget -O SPY_data.csv "https://raw.githubusercontent.com/Newzil-git/Financial-Risk-Management/main/FR%20Code/Part%201/1.6_market_data.csv"
file_path_aapl = "AAPL_data.csv"
file_path_spy = "SPY_data.csv"

## 1.3 导入必要的库

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False    # 用来正常显示负号

## 1.4 数据加载和预处理

In [ ]:
# 加载数据
try:
    spy_data = pd.read_csv(file_path_spy)
    aapl_data = pd.read_csv(file_path_aapl)
    print("数据加载成功")
except Exception as e:
    print(f"数据加载失败: {e}")
    # 如果从Google Drive加载失败，尝试从当前目录加载
    try:
        spy_data = pd.read_csv('S&P_500（B）_data.csv')
        aapl_data = pd.read_csv('AAPL_data.csv')
        print("从当前目录加载数据成功")
    except Exception as e:
        print(f"数据加载再次失败: {e}")

In [ ]:
# 数据预览
print("AAPL数据预览:")
display(aapl_data.head())
print("\nS&P 500数据预览:")
display(spy_data.head())

In [ ]:
# 数据清理，去掉无效的行（空值或无效数据）
aapl_data_clean = aapl_data[pd.to_numeric(aapl_data['Close'], errors='coerce').notnull()].reset_index(drop=True)
spy_data_clean = spy_data[pd.to_numeric(spy_data['Close'], errors='coerce').notnull()].reset_index(drop=True)

# 提取收盘价数据
spy_close_clean = spy_data_clean['Close'].dropna().astype(float)
aapl_close_clean = aapl_data_clean['Close'].dropna().astype(float)

# 查看清理后的数据统计
print("清理后AAPL数据统计:")
display(aapl_close_clean.describe())
print("\n清理后S&P 500数据统计:")
display(spy_close_clean.describe())

## 1.5 计算信息比率

In [ ]:
# 计算每日收益率（对数收益率）
spy_returns = np.log(spy_close_clean / spy_close_clean.shift(1)).dropna()
aapl_returns = np.log(aapl_close_clean / aapl_close_clean.shift(1)).dropna()

# 收益率预览
print("AAPL日收益率:")
display(aapl_returns.head())
print("\nS&P 500日收益率:")
display(spy_returns.head())

In [ ]:
# 计算平均收益率和标准差
avg_spy = spy_returns.mean()
avg_aapl = aapl_returns.mean()
std_spy = spy_returns.std()
std_aapl = aapl_returns.std()

# 计算信息比率
# 信息比率 = (投资组合收益率 - 基准收益率) / 跟踪误差
# 跟踪误差是投资组合相对于基准的超额收益的标准差
tracking_error = np.sqrt(std_aapl**2 + std_spy**2)  # 简化计算，实际应使用超额收益的标准差
ir = (avg_aapl - avg_spy) / tracking_error

# 显示计算结果
print(f"AAPL平均日收益率: {avg_aapl:.6f}")
print(f"S&P 500平均日收益率: {avg_spy:.6f}")
print(f"AAPL波动率: {std_aapl:.6f}")
print(f"S&P 500波动率: {std_spy:.6f}")
print(f"跟踪误差: {tracking_error:.6f}")
print(f"信息比率: {ir:.6f}")

In [ ]:
# 生成结果表格
performance_table = pd.DataFrame({
    '平均收益率': [avg_aapl, avg_spy],
    '波动率': [std_aapl, std_spy],
    '信息比率': [ir, float('nan')]
}, index=['投资组合P (AAPL)', '基准B (S&P 500)'])

display(performance_table)

## 1.6 结果可视化

In [ ]:
# 可视化价格走势
plt.figure(figsize=(12, 6))

# 创建两个Y轴
ax1 = plt.gca()
ax2 = ax1.twinx()

# 绘制AAPL价格走势
ax1.plot(aapl_close_clean.values, 'b-', label='AAPL价格')
ax1.set_xlabel('交易日')
ax1.set_ylabel('AAPL价格 ($)', color='b')
ax1.tick_params('y', colors='b')

# 绘制S&P 500价格走势
ax2.plot(spy_close_clean.values, 'r-', label='S&P 500价格')
ax2.set_ylabel('S&P 500价格 ($)', color='r')
ax2.tick_params('y', colors='r')

# 设置图例和标题
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.title('AAPL与S&P 500价格走势对比')
plt.grid(True)
plt.show()

In [ ]:
# 可视化收益率分布
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.hist(aapl_returns, bins=50, alpha=0.6, color='blue')
plt.axvline(avg_aapl, color='red', linestyle='dashed', linewidth=2)
plt.title('AAPL日收益率分布')
plt.xlabel('收益率')
plt.ylabel('频率')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.hist(spy_returns, bins=50, alpha=0.6, color='red')
plt.axvline(avg_spy, color='blue', linestyle='dashed', linewidth=2)
plt.title('S&P 500日收益率分布')
plt.xlabel('收益率')
plt.ylabel('频率')
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# 可视化超额收益
# 计算每日超额收益
min_length = min(len(aapl_returns), len(spy_returns))
excess_returns = aapl_returns[:min_length].values - spy_returns[:min_length].values

plt.figure(figsize=(12, 6))
plt.plot(excess_returns, color='green')
plt.axhline(y=0, color='black', linestyle='-')
plt.axhline(y=excess_returns.mean(), color='red', linestyle='--', label=f'平均超额收益: {excess_returns.mean():.6f}')
plt.title('AAPL相对于S&P 500的每日超额收益')
plt.xlabel('交易日')
plt.ylabel('超额收益')
plt.legend()
plt.grid(True)
plt.show()

## 1.7 结果分析与解释

通过上述计算和可视化，我们可以得出以下结论：

1. **信息比率解读**：
   - 计算得到的信息比率为{ir:.4f}
   - 信息比率大于0表示AAPL相对于S&P 500指数有超额收益
   - 信息比率的大小反映了投资组合经理在单位追踪误差下获取超额收益的能力

2. **收益率分析**：
   - AAPL的平均日收益率为{avg_aapl:.6f}，而S&P 500的平均日收益率为{avg_spy:.6f}
   - 这表明AAPL相对于市场基准有{(avg_aapl-avg_spy):.6f}的超额收益率

3. **波动率分析**：
   - AAPL的波动率为{std_aapl:.6f}，而S&P 500的波动率为{std_spy:.6f}
   - 这表明AAPL的风险水平{'高于' if std_aapl > std_spy else '低于'}市场基准

4. **超额收益的一致性**：
   - 通过超额收益图表，我们可以观察AAPL相对于S&P 500的表现是否一致
   - 图表显示的波动表明投资组合的超额收益并非时刻保持一致

5. **投资建议**：
   - 根据信息比率的结果，投资者可以评估是否值得投资于AAPL而非S&P 500指数
   - 信息比率需要结合其他指标一起考虑，如夏普比率、特雷诺比率等